In [ ]:
# Necessary libraries
import numpy as np
import marimo as mo
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Sklearn preprocessing libraries
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    RandomizedSearchCV,
)


# Regression libraries
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


# Y-data profinling
from ydata_profiling import ProfileReport


# metrices
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# ignore warnings
import warnings

warnings.filterwarnings("ignore")


# logging
import logging
from pathlib import Path


np.random.seed(42)

In [ ]:
dataset = pd.read_csv("data/raw/instagram_usage_lifestyle.csv")

In [ ]:
df = dataset.sample(frac=0.00001, random_state=42).reset_index(drop=True)

In [ ]:
df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"user_id\":336337,\"app_name\":\"Instagram\",\"age\":43,\"gender\":\"Male\",\"country\":\"Brazil\",\"urban_rural\":\"Rural\",\"income_level\":\"Low\",\"employment_status\":\"Student\",\"education_level\":\"Some college\",\"relationship_status\":\"Single\",\"has_children\":\"No\",\"exercise_hours_per_week\":1.3,\"sleep_hours_per_night\":6.2,\"diet_quality\":\"Average\",\"smoking\":\"Former\",\"alcohol_frequency\":\"Several times a week\",\"perceived_stress_score\":31,\"self_reported_happiness\":10,\"body_mass_index\":31.1,\"blood_pressure_systolic\":109,\"blood_pressure_diastolic\":78,\"daily_steps_count\":7938,\"weekly_work_hours\":26.6,\"hobbies_count\":3,\"social_events_per_month\":2,\"books_read_per_year\":10,\"volunteer_hours_per_month\":4.3,\"travel_frequency_per_year\":5,\"daily_active_minutes_instagram\":201.0,\"sessions_per_day\":9,\"posts_created_per_week\":6,\"reels_watched_per_day\":176,\"stories_viewed_per_day\":103,\"likes_given_per_day\":133,\"comments_written_per_day\":37,\"dms_sent_per_week\":22,\"dms_received_per_week\":26,\"ads_viewed_per_day\":20,\"ads_clicked_per_day\":7,\"time_on_feed_per_day\":105,\"time_on_explore_per_day\":25,\"time_on_messages_per_day\":47,\"time_on_reels_per_day\":56,\"followers_count\":292,\"following_count\":478,\"uses_premium_features\":\"Yes\",\"notification_response_rate\":0.64,\"account_creation_year\":2011,\"last_login_date\":\"2025-02-11\",\"average_session_length_minutes\":22.3},{\"user_id\":253653,\"app_name\":\"Instagram\",\"age\":17,\"gender\":\"Male\",\"country\":\"Germany\",\"urban_rural\":\"Urban\",\"income_level\":\"Low\",\"employment_status\":\"Student\",\"education_level\":\"Some college\",\"relationship_status\":\"Single\",\"has_children\":\"Yes\",\"exercise_hours_per_week\":18.1,\"sleep_hours_per_night\":8.0,\"diet_quality\":\"Average\",\"smoking\":\"No\",\"alcohol_frequency\":\"Never\",\"perceived_stress_score\":25,\"self_reported_happiness\":10,\"body_mass_index\":18.2,\"blood_pressure_systolic\":100,\"blood_pressure_diastolic\":62,\"daily_steps_count\":7995,\"weekly_work_hours\":28.3,\"hobbies_count\":2,\"social_events_per_month\":4,\"books_read_per_year\":14,\"volunteer_hours_per_month\":6.4,\"travel_frequency_per_year\":1,\"daily_active_minutes_instagram\":222.0,\"sessions_per_day\":7,\"posts_created_per_week\":14,\"reels_watched_per_day\":238,\"stories_viewed_per_day\":115,\"likes_given_per_day\":139,\"comments_written_per_day\":39,\"dms_sent_per_week\":37,\"dms_received_per_week\":46,\"ads_viewed_per_day\":18,\"ads_clicked_per_day\":6,\"time_on_feed_per_day\":100,\"time_on_explore_per_day\":61,\"time_on_messages_per_day\":32,\"time_on_reels_per_day\":70,\"followers_count\":1359,\"following_count\":1232,\"uses_premium_features\":\"No\",\"notification_response_rate\":0.33,\"account_creation_year\":2020,\"last_login_date\":\"2025-09-20\",\"average_session_length_minutes\":31.7},{\"user_id\":1079086,\"app_name\":\"Instagram\",\"age\":52,\"gender\":\"Female\",\"country\":\"United Kingdom\",\"urban_rural\":\"Urban\",\"income_level\":\"Middle\",\"employment_status\":\"Full-time employed\",\"education_level\":\"High school\",\"relationship_status\":\"Married\",\"has_children\":\"No\",\"exercise_hours_per_week\":4.6,\"sleep_hours_per_night\":7.9,\"diet_quality\":\"Average\",\"smoking\":\"No\",\"alcohol_frequency\":\"Rarely\",\"perceived_stress_score\":36,\"self_reported_happiness\":10,\"body_mass_index\":27.9,\"blood_pressure_systolic\":157,\"blood_pressure_diastolic\":96,\"daily_steps_count\":7817,\"weekly_work_hours\":46.1,\"hobbies_count\":1,\"social_events_per_month\":5,\"books_read_per_year\":11,\"volunteer_hours_per_month\":1.8,\"travel_frequency_per_year\":4,\"daily_active_minutes_instagram\":197.0,\"sessions_per_day\":11,\"posts_created_per_week\":1,\"reels_watched_per_day\":166,\"stories_viewed_per_day\":118,\"likes_given_per_day\":143,\"comments_written_per_day\":38,\"dms_sent_per_week\"

In [ ]:
# profile = ProfileReport(
#     df=dataset,
#     title="Effect of Social Media (Instagram) in Human Happiness",
#     explorative=True,
# )

# profile.to_file("ydata.html")

**Decisions based on y-data profiling:**

- Have to drop *user_id, app__name, last_login_date* features.
- Have to impute the missing values in the *perceived_stress_score, hobbies_count, social_events_per_month, travel_frequency_per_year, posts_created_per_week, ads_clicked_per_day, linked_accounts_count* features.
- Have to convert the categorical values into numerical values

In [ ]:
# Drop the necessary features

if "user_id" in df.columns and "app_name" in df.columns:
    df.drop(columns=["user_id", "app_name"], inplace=True)

In [ ]:
if "last_login_date" in df.columns:
    df.drop(columns=["last_login_date"], inplace=True)

In [ ]:
if (
    "user_id" not in df.columns
    and "app_name" not in df.columns
    and "last_login_date" not in df.columns
):
    print("Dropping done!")
else:
    print("Error occurred while dropping!")

Dropping done!


In [ ]:
# for col in df.columns:
#     print(col, df[col].dtype)

In [ ]:
numerical_features = X.select_dtypes(
    include=["int64", "float64"]
).columns.tolist()
categorical_features = X.select_dtypes(
    include=["object", "category"]
).columns.tolist()

In [ ]:
mo.hstack(
    [
        numerical_features,
        categorical_features,
    ]
)

<marimo-json-output data-json-data='["age","exercise_hours_per_week","sleep_hours_per_night","perceived_stress_score","body_mass_index","blood_pressure_systolic","blood_pressure_diastolic","daily_steps_count","weekly_work_hours","hobbies_count","social_events_per_month","books_read_per_year","volunteer_hours_per_month","travel_frequency_per_year","daily_active_minutes_instagram","sessions_per_day","posts_created_per_week","reels_watched_per_day","stories_viewed_per_day","likes_given_per_day","comments_written_per_day","dms_sent_per_week","dms_received_per_week","ads_viewed_per_day","ads_clicked_per_day","time_on_feed_per_day","time_on_explore_per_day","time_on_messages_per_day","time_on_reels_per_day","followers_count","following_count","notification_response_rate","account_creation_year","average_session_length_minutes","linked_accounts_count","user_engagement_score"]' data-value-types='"python"'> <marimo-json-output data-json-data='["gender","country","urban_rural","income_level","employment_status","education_level","relationship_status","has_children","diet_quality","smoking","alcohol_frequency","uses_premium_features","content_type_preference","preferred_content_theme","privacy_setting_level","two_factor_auth_enabled","biometric_login_used","subscription_status"]' data-value-types='"python"'>

In [ ]:
nominal_features = [
    "gender",
    "country",
    "urban_rural",
    "employment_status",
    "relationship_status",
    "has_children",
    "smoking",
    "alcohol_frequency",
    "uses_premium_features",
    "content_type_preference",
    "preferred_content_theme",
    "privacy_setting_level",
    "two_factor_auth_enabled",
    "biometric_login_used",
    "subscription_status",
]

ordinal_features = ["income_level", "education_level", "diet_quality"]

In [ ]:
target_feature = "self_reported_happiness"

**Imputing the missing values for both numerical and categorical features**

In [ ]:
# For numerical features

numerical_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

In [ ]:
# For nominal categorical features (constant -> most frequent)

nominal_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        (
            "encoder",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
        ),
    ]
)

In [ ]:
# For ordinal categorical features (constant -> most frequent)

ordinal_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        (
            "encoder",
            OrdinalEncoder(
                categories=[
                    ["Low", "Lower-middle", "Upper-middle", "Middle", "High"],
                    [
                        "High school",
                        "Some college",
                        "Bachelor’s",
                        "Master’s",
                        "Other",
                    ],
                    ["Very poor", "Poor", "Average", "Good", "Excellent"],
                ],
                handle_unknown="use_encoded_value",
                unknown_value=-1,
            ),
        ),
    ]
)

In [ ]:
# Combine all pipelines

preprocessor = ColumnTransformer(
    [
        ("numerical", numerical_pipeline, numerical_features),
        ("nominal", nominal_pipeline, nominal_features),
        ("ordinal", ordinal_pipeline, ordinal_features),
    ]
)

In [ ]:
X = df.drop(target_feature, axis=1)
y = df[target_feature]

In [ ]:
# Train and test split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [ ]:
# Base learners

lr_reg = LinearRegression()
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
gb_reg = GradientBoostingRegressor(n_estimators=100, random_state=42)

In [ ]:
# Voting regressor

voting_reg = VotingRegressor(
    estimators=[
        ("lr_reg", lr_reg),
        ("rf_reg", rf_reg),
        ("gb_reg", gb_reg),
    ]
)

In [ ]:
# Stacking regressor

stacking_reg = StackingRegressor(
    estimators=[
        ("lr_reg", lr_reg),
        ("rf_reg", rf_reg),
        ("gb_reg", gb_reg),
    ],
    final_estimator=Ridge(),  # meta learner
)

**Model Training**

In [ ]:
# Dictionary of all models

model_to_train = {
    "Linear Regression": lr_reg,
    "Random Forest Regression": rf_reg,
    "Gradient Boosting Regression": gb_reg,
    "Voting Ensemble": voting_reg,
    "Stacking Ensemble": stacking_reg,
}

**Model Training and Evaluation**

In [ ]:
try:
    logger.info("Starting baseline model training (without PCA)...")
    results = []

    for _name, _model in model_to_train.items():
        try:
            logger.info(f"Training {_name} without PCA...")

            # Full pipeline with model

            full_pipeline = Pipeline(
                steps=[("preprocessor", preprocessor), ("model", _model)]
            )

            # Training
            full_pipeline.fit(X_train, y_train)

            # Prediction
            _y_pred = full_pipeline.predict(X_test)

            # Evaluation

            _r2 = r2_score(y_test, _y_pred)
            _mae = mean_absolute_error(y_test, _y_pred)
            _mse = mean_squared_error(y_test, _y_pred)
            _rmse = np.sqrt(_mse)

            results.append(
                {"Model": _name, "R2 Score": _r2, "MAE": _mae, "RMSE": _rmse}
            )

        except Exception as e:
            logger.error(
                f"Error training {_name} with PCA: {str(e)}", exc_info=True
            )
            continue

    results_df = pd.DataFrame(results).sort_values("R2 Score", ascending=False)

except Exception as e:
    logger.error(f"Critical error in PCA training: {str(e)}", exc_info=True)

2026-01-25 07:42:18,813 | INFO | crimson-nebula | Starting baseline model training (without PCA)...
2026-01-25 07:42:18,814 | INFO | crimson-nebula | Training Linear Regression without PCA...
2026-01-25 07:42:18,859 | INFO | crimson-nebula | Training Random Forest Regression without PCA...
2026-01-25 07:42:19,063 | INFO | crimson-nebula | Training Gradient Boosting Regression without PCA...
2026-01-25 07:42:19,167 | INFO | crimson-nebula | Training Voting Ensemble without PCA...
2026-01-25 07:42:19,375 | INFO | crimson-nebula | Training Stacking Ensemble without PCA...


In [ ]:
# Creating logs directory
log_dir = Path("logs")
log_dir.mkdir(exist_ok=True)

log_file = log_dir / "crimson_nebula.log"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.FileHandler(log_file), logging.StreamHandler()],
    force=True,
)

logger = logging.getLogger("crimson-nebula")
logger.info("Logging is set up.")
logging.info("crimson-nebula - Model Training Started.")

2026-01-25 07:42:18,781 | INFO | crimson-nebula | Logging is set up.
2026-01-25 07:42:18,786 | INFO | root | crimson-nebula - Model Training Started.


In [ ]:
results_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"\":2,\"Model\":\"Gradient Boosting Regression\",\"R2 Score\":0.537707338271659,\"MAE\":1.7252455828211037,\"RMSE\":2.039763210658303},{\"\":3,\"Model\":\"Voting Ensemble\",\"R2 Score\":0.44224028333815246,\"MAE\":2.12570178427427,\"RMSE\":2.2404993751297115},{\"\":1,\"Model\":\"Random Forest Regression\",\"R2 Score\":0.2841694444444445,\"MAE\":2.4175,\"RMSE\":2.5382031045603894},{\"\":0,\"Model\":\"Linear Regression\",\"R2 Score\":0.01694102550827259,\"MAE\":2.5017306366992207,\"RMSE\":2.974479915955989},{\"\":4,\"Model\":\"Stacking Ensemble\",\"R2 Score\":-0.15810814221564118,\"MAE\":3.0956788565001254,\"RMSE\":3.2284629903315865}]"' data-total-rows='5' data-total-columns='4' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["Model",["string","object"]],["R2 Score",["number","float64"]],["MAE",["number","float64"]],["RMSE",["number","float64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-show-data-types='true' data-show-page-size-selector='false' data-show-column-explorer='true' data-show-chart-builder='true' data-row-headers='[["",["integer","int64"]]]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

**Principal Component Analysis**

In [ ]:
# Transform the data using preprocessor

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [ ]:
# Apply principal component analysis

pca = PCA()

X_train_pca = pca.fit_transform(X_train_transformed)
X_test_pca = pca.transform(X_test_transformed)

In [ ]:
# Explained variance ratio

explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

In [ ]:
# Plot explained variance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Individual explained variance
ax1.bar(range(1, len(explained_variance) + 1), explained_variance)
ax1.set_xlabel("Principal Component")
ax1.set_ylabel("Explained Variance Ratio")
ax1.set_title("PCA - Explained Variance by Component")

# Cumulative explained variance
ax2.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, "bo-")
ax2.axhline(y=0.95, color="r", linestyle="--", label="95% Variance")
ax2.set_xlabel("Number of Components")
ax2.set_ylabel("Cumulative Explained Variance")
ax2.set_title("PCA - Cumulative Explained Variance")
ax2.legend()
ax2.grid(True)

plt.tight_layout()

In [ ]:
# Find number of components for 95% variance
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1

mo.md(f"**Number of components needed for 95% variance: {n_components_95}**")

<span class="markdown prose dark:prose-invert contents"><span class="paragraph"><strong>Number of components needed for 95% variance: 9</strong></span></span>

In [ ]:
# PCA with optimal number of components
pca_optimal = PCA(n_components=n_components_95)

In [ ]:
try:
    logger.info(
        f"Starting PCA-based model training with {n_components_95} components..."
    )

    results_with_pca = []

    for _name, _model in model_to_train.items():
        try:
            logger.info(f"Training {_name} with PCA...")
            # Pipeline with PCA
            full_pipeline_pca = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("pca", PCA(n_components=n_components_95)),
                    ("model", _model),
                ]
            )

            # Training
            full_pipeline_pca.fit(X_train, y_train)

            # Prediction
            _y_pred = full_pipeline_pca.predict(X_test)

            # Evaluation
            _r2 = r2_score(y_test, _y_pred)
            _mae = mean_absolute_error(y_test, _y_pred)
            _mse = mean_squared_error(y_test, _y_pred)
            _rmse = np.sqrt(_mse)

            logger.info(
                f"{_name} with PCA - R2: {_r2:.4f}, MAE: {_mae:.4f}, RMSE: {_rmse:.4f}"
            )

            results_with_pca.append(
                {"Model": _name, "R2 Score": _r2, "MAE": _mae, "RMSE": _rmse}
            )
        except Exception as e:
            logger.error(
                f"Error training {_name} with PCA: {str(e)}", exc_info=True
            )
            continue

    results_with_pca_df = pd.DataFrame(results_with_pca).sort_values(
        "R2 Score", ascending=False
    )

except Exception as e:
    logger.critical(f"Critical error in PCA training: {str(e)}", exc_info=True)
    raise

2026-01-25 07:42:21,268 | INFO | crimson-nebula | Starting PCA-based model training with 9 components...
2026-01-25 07:42:21,269 | INFO | crimson-nebula | Training Linear Regression with PCA...
2026-01-25 07:42:21,307 | INFO | crimson-nebula | Linear Regression with PCA - R2: -0.0637, MAE: 2.6763, RMSE: 3.0941
2026-01-25 07:42:21,308 | INFO | crimson-nebula | Training Random Forest Regression with PCA...
2026-01-25 07:42:21,503 | INFO | crimson-nebula | Random Forest Regression with PCA - R2: 0.4741, MAE: 1.9075, RMSE: 2.1757
2026-01-25 07:42:21,503 | INFO | crimson-nebula | Training Gradient Boosting Regression with PCA...
2026-01-25 07:42:21,579 | INFO | crimson-nebula | Gradient Boosting Regression with PCA - R2: 0.7593, MAE: 1.2934, RMSE: 1.4719
2026-01-25 07:42:21,580 | INFO | crimson-nebula | Training Voting Ensemble with PCA...
2026-01-25 07:42:21,759 | INFO | crimson-nebula | Voting Ensemble with PCA - R2: 0.6617, MAE: 1.5207, RMSE: 1.7448
2026-01-25 07:42:21,759 | INFO | crims

In [ ]:
results_with_pca_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"\":2,\"Model\":\"Gradient Boosting Regression\",\"R2 Score\":0.759271968945683,\"MAE\":1.2933995818801527,\"RMSE\":1.4719212884827955},{\"\":3,\"Model\":\"Voting Ensemble\",\"R2 Score\":0.6617336678599051,\"MAE\":1.5207300951927791,\"RMSE\":1.7448200449504399},{\"\":1,\"Model\":\"Random Forest Regression\",\"R2 Score\":0.4740527777777779,\"MAE\":1.9075,\"RMSE\":2.1756665645268347},{\"\":0,\"Model\":\"Linear Regression\",\"R2 Score\":-0.06370786745737278,\"MAE\":2.6762726574663,\"RMSE\":3.094086425282325},{\"\":4,\"Model\":\"Stacking Ensemble\",\"R2 Score\":-3.8067170536477786,\"MAE\":6.105831677680579,\"RMSE\":6.5772679345477485}]"' data-total-rows='5' data-total-columns='4' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["Model",["string","object"]],["R2 Score",["number","float64"]],["MAE",["number","float64"]],["RMSE",["number","float64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-show-data-types='true' data-show-page-size-selector='false' data-show-column-explorer='true' data-show-chart-builder='true' data-row-headers='[["",["integer","int64"]]]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
# Hyperparameter grids for each _model

param_grids = {
    "Random Forest Regression": {
        "model__n_estimators": [50, 100, 200],
        "model__max_depth": [10, 20, 30, None],
        "model__min_samples_split": [2, 5, 10],
        "model__min_samples_leaf": [1, 2, 4],
    },
    "Gradient Boosting Regression": {
        "model__n_estimators": [50, 100, 200],
        "model__learning_rate": [0.01, 0.1, 0.2],
        "model__max_depth": [3, 5, 7],
        "model__subsample": [0.8, 0.9, 1.0],
    },
    "Linear Regression": {},  # No hyperparameters to tune
    "Voting Ensemble": {
        "model__rf_reg__n_estimators": [50, 100],
        "model__gb_reg__n_estimators": [50, 100],
        "model__gb_reg__learning_rate": [0.01, 0.1],
    },
    "Stacking Ensemble": {
        "model__rf_reg__n_estimators": [50, 100],
        "model__gb_reg__n_estimators": [50, 100],
        "model__final_estimator__alpha": [0.1, 1.0, 10.0],
    },
}

In [ ]:
# For RandomizedSearchCV - larger search space

param_distributions = {
    "Random Forest Regression": {
        "model__n_estimators": [50, 100, 150, 200, 300],
        "model__max_depth": [5, 10, 15, 20, 25, 30, None],
        "model__min_samples_split": [2, 5, 10, 15],
        "model__min_samples_leaf": [1, 2, 4, 6],
        "model__max_features": ["sqrt", "log2", None],
    },
    "Gradient Boosting Regression": {
        "model__n_estimators": [50, 100, 150, 200, 300],
        "model__learning_rate": [0.01, 0.05, 0.1, 0.15, 0.2],
        "model__max_depth": [3, 4, 5, 6, 7, 8],
        "model__subsample": [0.7, 0.8, 0.9, 1.0],
        "model__min_samples_split": [2, 5, 10],
    },
}

In [ ]:
try:
    logger.info("Starting GridSearchCV hyperparameter tuning...")

    grid_search_results = []
    best_models_grid = {}

    for _name, _model in model_to_train.items():
        if _name in param_grids and param_grids[_name]:
            try:
                logging.info(f"\nTunning {_name} with GridSearchCV...")

                # Create pipeline
                _pipeline = Pipeline(
                    [("preprocessor", preprocessor), ("model", _model)]
                )
                grid_search = GridSearchCV(
                    _pipeline,
                    param_grids[_name],
                    cv=5,
                    scoring="r2",
                    n_jobs=-1,
                    verbose=1,
                )

                # Fit
                grid_search.fit(X_train, y_train)

                # Best _model
                best_models_grid[_name] = grid_search.best_estimator_

                # Predict
                _y_pred = grid_search.predict(X_test)

                # Evaluate
                _r2 = r2_score(y_test, _y_pred)
                _mae = mean_absolute_error(y_test, _y_pred)
                _mse = mean_squared_error(y_test, _y_pred)
                _rmse = np.sqrt(_mse)

                grid_search_results.append(
                    {
                        "_model": _name,
                        "Best Params": str(grid_search.best_params_),
                        "R2 Score": _r2,
                        "MAE": _mae,
                        "RMSE": _rmse,
                    }
                )

            except Exception as e:
                logger.error(
                    f"Error in GridSearchCV for {_name}: {str(e)}",
                    exc_info=True,
                )
                continue
        else:
            logger.info(
                f"Skipping GridSearchCV for {_name} - no hyperparameters to tune"
            )

    grid_results_df = pd.DataFrame(grid_search_results).sort_values(
        "R2 Score", ascending=False
    )
    logger.info("GridSearchCV tuning completed successfully")

except Exception as e:
    logger.critical(f"Critical error in GridSearchCV: {str(e)}", exc_info=True)
    raise

2026-01-25 07:42:22,737 | INFO | crimson-nebula | Starting GridSearchCV hyperparameter tuning...
2026-01-25 07:42:22,738 | INFO | crimson-nebula | Skipping GridSearchCV for Linear Regression - no hyperparameters to tune
2026-01-25 07:42:22,738 | INFO | root | 
Tunning Random Forest Regression with GridSearchCV...


Fitting 5 folds for each of 108 candidates, totalling 540 fits


2026-01-25 07:43:06,795 | INFO | root | 
Tunning Gradient Boosting Regression with GridSearchCV...


Fitting 5 folds for each of 81 candidates, totalling 405 fits


2026-01-25 07:43:26,358 | INFO | root | 
Tunning Voting Ensemble with GridSearchCV...


Fitting 5 folds for each of 8 candidates, totalling 40 fits


2026-01-25 07:43:29,696 | INFO | root | 
Tunning Stacking Ensemble with GridSearchCV...


Fitting 5 folds for each of 12 candidates, totalling 60 fits


2026-01-25 07:43:52,373 | INFO | crimson-nebula | GridSearchCV tuning completed successfully


In [ ]:
grid_results_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"\":0,\"_model\":\"Random Forest Regression\",\"Best Params\":\"{'model__max_depth': 10, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}\",\"R2 Score\":0.2996895833333333,\"MAE\":2.38375,\"RMSE\":2.510536546238672},{\"\":2,\"_model\":\"Voting Ensemble\",\"Best Params\":\"{'model__gb_reg__learning_rate': 0.01, 'model__gb_reg__n_estimators': 50, 'model__rf_reg__n_estimators': 100}\",\"R2 Score\":0.16942697573693477,\"MAE\":2.4769344425911477,\"RMSE\":2.7340733747227026},{\"\":1,\"_model\":\"Gradient Boosting Regression\",\"Best Params\":\"{'model__learning_rate': 0.2, 'model__max_depth': 3, 'model__n_estimators': 50, 'model__subsample': 0.8}\",\"R2 Score\":0.1543879636376222,\"MAE\":2.295576174551014,\"RMSE\":2.7587149775323656},{\"\":3,\"_model\":\"Stacking Ensemble\",\"Best Params\":\"{'model__final_estimator__alpha': 10.0, 'model__gb_reg__n_estimators': 50, 'model__rf_reg__n_estimators': 50}\",\"R2 Score\":-0.04554860919705783,\"MAE\":2.8166131650790733,\"RMSE\":3.067562140001979}]"' data-total-rows='4' data-total-columns='5' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["_model",["string","object"]],["Best Params",["string","object"]],["R2 Score",["number","float64"]],["MAE",["number","float64"]],["RMSE",["number","float64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-show-data-types='true' data-show-page-size-selector='false' data-show-column-explorer='true' data-show-chart-builder='true' data-row-headers='[["",["integer","int64"]]]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
random_search_results = []
best_models_random = {}

try:
    logger.info("Starting RandomizedSearchCV hyperparameter tuning...")

    for _name, _model in model_to_train.items():
        if _name in param_distributions:
            try:
                logging.info(f"\nTunning {_name} with RandomizedSearchCV...")

                # Create pipeline
                _pipeline = Pipeline(
                    [("preprocessor", preprocessor), ("model", _model)]
                )
                random_search = RandomizedSearchCV(
                    _pipeline,
                    param_distributions[_name],
                    n_iter=20,
                    cv=5,
                    scoring="r2",
                    n_jobs=-1,
                    verbose=1,
                    random_state=42,
                )

                # Fit
                random_search.fit(X_train, y_train)

                # Best model
                best_models_random[_name] = random_search.best_estimator_

                # Predict
                _y_pred = random_search.predict(X_test)

                # Evaluate
                _r2 = r2_score(y_test, _y_pred)
                _mae = mean_absolute_error(y_test, _y_pred)
                _mse = mean_squared_error(y_test, _y_pred)
                _rmse = np.sqrt(_mse)

                random_search_results.append(
                    {
                        "Model": _name,
                        "Best Params": str(random_search.best_params_),
                        "R2 Score": _r2,
                        "MAE": _mae,
                        "RMSE": _rmse,
                    }
                )
            except Exception as e:
                logger.error(
                    f"Error in RandomizedSearchCV for {_name}: {str(e)}",
                    exc_info=True,
                )
                continue
        else:
            print(f"Skipping {_name} - no hyperparameters to tune")

    random_results_df = pd.DataFrame(random_search_results).sort_values(
        "R2 Score", ascending=False
    )
    logger.info("RandomizedSearchCV tuning completed successfully")
except Exception as e:
    logger.critical(
        f"Critical error in RandomizedSearchCV: {str(e)}", exc_info=True
    )
    raise

2026-01-25 07:43:52,436 | INFO | crimson-nebula | Starting RandomizedSearchCV hyperparameter tuning...


Skipping Linear Regression - no hyperparameters to tune


2026-01-25 07:43:52,436 | INFO | root | 
Tunning Random Forest Regression with RandomizedSearchCV...


Fitting 5 folds for each of 20 candidates, totalling 100 fits


2026-01-25 07:44:02,758 | INFO | root | 
Tunning Gradient Boosting Regression with RandomizedSearchCV...


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Skipping Voting Ensemble - no hyperparameters to tune
Skipping Stacking Ensemble - no hyperparameters to tune


2026-01-25 07:44:08,269 | INFO | crimson-nebula | RandomizedSearchCV tuning completed successfully


In [ ]:
random_results_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"\":1,\"Model\":\"Gradient Boosting Regression\",\"Best Params\":\"{'model__subsample': 0.8, 'model__n_estimators': 50, 'model__min_samples_split': 2, 'model__max_depth': 6, 'model__learning_rate': 0.1}\",\"R2 Score\":0.45086564044922506,\"MAE\":2.133892853583106,\"RMSE\":2.2231080126608727},{\"\":0,\"Model\":\"Random Forest Regression\",\"Best Params\":\"{'model__n_estimators': 150, 'model__min_samples_split': 2, 'model__min_samples_leaf': 1, 'model__max_features': 'sqrt', 'model__max_depth': 15}\",\"R2 Score\":0.4100407407407407,\"MAE\":2.1016666666666666,\"RMSE\":2.304264163097047}]"' data-total-rows='2' data-total-columns='5' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["Model",["string","object"]],["Best Params",["string","object"]],["R2 Score",["number","float64"]],["MAE",["number","float64"]],["RMSE",["number","float64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-show-data-types='true' data-show-page-size-selector='false' data-show-column-explorer='true' data-show-chart-builder='true' data-row-headers='[["",["integer","int64"]]]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

**Choose best model**

In [ ]:
grid_results_df["Method"] = "GridSearchCV"
random_results_df["Method"] = "RandomizedSearchCV"
results_with_pca_df["Method"] = "PCA"

In [ ]:
# Combine all results
all_results = pd.concat(
    [grid_results_df, random_results_df, results_with_pca_df],
    ignore_index=True,
)

In [ ]:
all_results_sorted = all_results.sort_values("R2 Score", ascending=False)

In [ ]:
best_model_info = all_results_sorted.iloc[0]

In [ ]:
best_model_info

/home/abdullahalazmi/Programming/crimson-nebula/.venv/lib64/python3.10/site-packages/marimo/_output/formatters/pandas_formatters.py:46: UserWarning: DataFrame has integer column names. This is not supported and can lead to bugs. Please use strings for column names.
  return table(


<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"\":\"_model\",\"6\":NaN},{\"\":\"Best Params\",\"6\":NaN},{\"\":\"R2 Score\",\"6\":0.759271968945683},{\"\":\"MAE\",\"6\":1.2933995818801527},{\"\":\"RMSE\",\"6\":1.4719212884827955},{\"\":\"Method\",\"6\":\"PCA\"},{\"\":\"Model\",\"6\":\"Gradient Boosting Regression\"}]"' data-total-rows='7' data-total-columns='1' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["6",["string","object"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-show-data-types='true' data-show-page-size-selector='true' data-show-column-explorer='true' data-show-chart-builder='true' data-row-headers='[["",["string","object"]]]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

**Save the model**

In [ ]:
best_model_name = best_model_info["Model"]
best_method = best_model_info["Method"]

if best_method == "GridSearchCV":
    best_model = best_models_grid[best_model_name]
elif best_method == "RandomizedSearchCV":
    best_model = best_models_random[best_model_name]
else:
    best_model = model_to_train[best_model_name]

In [ ]:
import pickle
import json

try:
    # Create models directory
    models_dir = Path("src/models")
    models_dir.mkdir(parents=True, exist_ok=True)

    # Save the model
    model_path = models_dir / "best_model.pkl"
    with open(model_path, "wb") as f:
        pickle.dump(best_model, f)

    logger.info(f"Model saved successfully at {model_path}")

except Exception as e:
    logger.error(f"Error saving model: {str(e)}", exc_info=True)

2026-01-25 07:52:00,626 | INFO | crimson-nebula | Model saved successfully at src/models/best_model.pkl


In [ ]:
try:
    logger.info("Retraining best model on full dataset...")

    # Prepare full dataset
    X_full = dataset.drop(target_feature, axis=1)
    y_full = dataset[target_feature]

    # Extract the model based on the method used
    if best_method in ["GridSearchCV", "RandomizedSearchCV"]:
        # These are Pipeline objects
        model_only = best_model.named_steps["model"]
    else:
        # PCA method returns raw model
        model_only = best_model

    # Create full pipeline with preprocessor and model
    full_pipeline_retrain = Pipeline(
        steps=[("preprocessor", preprocessor), ("model", model_only)]
    )

    # Retrain the model on full dataset
    full_pipeline_retrain.fit(X_full, y_full)

    # Update best_model to be the full pipeline
    best_model_r = full_pipeline_retrain

    logger.info(
        "Model retrained successfully on full dataset with preprocessor"
    )

except Exception as e:
    logger.error(f"Error retraining model: {str(e)}", exc_info=True)

2026-01-25 07:52:57,944 | INFO | crimson-nebula | Retraining best model on full dataset...
2026-01-25 08:23:09,063 | INFO | crimson-nebula | Model retrained successfully on full dataset with preprocessor


In [ ]:
try:
    models_dir_r = Path("src/models")
    models_dir_r.mkdir(parents=True, exist_ok=True)

    # Save the retrained model
    model_path_r = models_dir_r / "crimson_nebula.pkl"
    with open(model_path_r, "wb") as file:
        pickle.dump(best_model_r, file)

    logger.info(f"Retrained model saved at {model_path_r}")

except Exception as e:
    logger.error(f"Error saving retrained model: {str(e)}", exc_info=True)

2026-01-25 08:25:59,817 | INFO | crimson-nebula | Retrained model saved at src/models/crimson_nebula.pkl
